In [3]:
import pandas as pd
import numpy as np
import os
import re

In [4]:
cd "C:\Users\82105\Desktop\프로젝트 7조\인스타그램 크롤링 코드"

c:\Users\82105\Desktop\프로젝트 7조\인스타그램 크롤링 코드


In [3]:
df = pd.read_csv("total.csv", engine='python', encoding='cp949')
df.head()

,site,restaurant,type,time,review,date
0,인스타그램,겐로쿠우동,라멘 / 소바 / 우동,11:30 - 21:30,겐로쿠우동대학로점 겐로쿠우동 니쿠우동 서울맛집 대학로맛집 혜화동맛집 우...,2021-11-09
1,네이버지도,겐로쿠우동,라멘 / 소바 / 우동,11:30 - 21:30,너무 맛있고 전통있는 맛집,2021-11-06
2,네이버지도,겐로쿠우동,라멘 / 소바 / 우동,11:30 - 21:30,조앙,2021-11-06
3,네이버지도,겐로쿠우동,라멘 / 소바 / 우동,11:30 - 21:30,고기 양에 비해서 너무 비싸요 면은 원가가 저렴하니까 추가해도 손해가 없어서 8...,2021-11-01
4,네이버지도,겐로쿠우동,라멘 / 소바 / 우동,11:30 - 21:30,직원이 불친절해요,2021-11-01


In [13]:
# review 열 기준 중복 행 제거
df = df.drop_duplicates(['review'])

In [14]:
# 2020.10.01 이전 행 삭제
df = df.query('"2020-10-01"<= date <= "2021-11-17"')
df

,site,restaurant,type,time,review,date
0,인스타그램,겐로쿠우동,라멘 / 소바 / 우동,11:30 - 21:30,겐로쿠우동대학로점 겐로쿠우동 니쿠우동 서울맛집 대학로맛집 혜화동맛집 우...,2021-11-09
1,네이버지도,겐로쿠우동,라멘 / 소바 / 우동,11:30 - 21:30,너무 맛있고 전통있는 맛집,2021-11-06
2,네이버지도,겐로쿠우동,라멘 / 소바 / 우동,11:30 - 21:30,조앙,2021-11-06
3,네이버지도,겐로쿠우동,라멘 / 소바 / 우동,11:30 - 21:30,고기 양에 비해서 너무 비싸요 면은 원가가 저렴하니까 추가해도 손해가 없어서 8...,2021-11-01
4,네이버지도,겐로쿠우동,라멘 / 소바 / 우동,11:30 - 21:30,직원이 불친절해요,2021-11-01
...,...,...,...,...,...,...
18987,인스타그램,혼가츠대학로점,NaN,NaN,퀄리티 좋은 대학로 돈까스 맛집 원래 돈까스를 좋아하기도 하고이 집이 진짜...,2021-02-12
18989,인스타그램,혼가츠대학로점,NaN,NaN,일상그램 댓글소통 일상생활 맞팔해요우리 선맞팔 일상사진 맞팔댓 일상스타그램 소소한...,2020-12-01
18990,인스타그램,혼가츠대학로점,NaN,NaN,혼가츠대학로점,2020-11-28
18993,인스타그램,혼가츠대학로점,NaN,NaN,치즈가 살아있다 사실 돈까스는 어딜가나 비슷하기 마련이어서 맛집이다...,2020-10-26


In [15]:
df.to_csv("FinalTotal.csv")

In [7]:
df = pd.read_csv("FinalTotal.csv", engine='python', encoding='cp949')

##### + 중간 전처리 과정 생략 (엑셀): 공백 행 제거 / 중국어 및 일본어 리뷰 행 제거 / 영어 리뷰 번역본 대체

In [9]:
df['review'].dropna(axis=0)

0         겐로쿠우동대학로점  겐로쿠우동  니쿠우동  서울맛집  대학로맛집  혜화동맛집  우...
1                                           너무 맛있고 전통있는 맛집
2                                                       조앙
3        고기 양에 비해서 너무 비싸요   면은 원가가 저렴하니까 추가해도 손해가 없어서 8...
4                                                직원이 불친절해요
                               ...                        
12347      퀄리티 좋은 대학로 돈까스 맛집    원래 돈까스를 좋아하기도 하고이 집이 진짜...
12348     일상그램 댓글소통 일상생활 맞팔해요우리 선맞팔 일상사진 맞팔댓 일상스타그램 소소한...
12349                                              혼가츠대학로점
12350       치즈가 살아있다      사실 돈까스는 어딜가나 비슷하기 마련이어서  맛집이다...
12351     혼가츠  서울 종로구 대명길 24 지하 1층  먹스타그램  맛스타그램  푸드스타그...
Name: review, Length: 12351, dtype: object

### 공백 뭉텅이 지우기

In [52]:
# 공백이 1번만 나오도록 대체 (정규표현식 '+'는 1번을 의미함)
def remove(str_content):
    return re.sub(' +', ' ', str_content)

In [53]:
for i in range(len(df['review'])):
    try:
        # iloc: 인덱스 번호로 행 추출
        original = df['review'].iloc[i] 
        rr = remove(original)
        df['review'].iloc[i] = rr
    except:
        pass

C:\Users\82105\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [51]:
df.to_csv("RealFinalTotal.csv")